In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library

<h2>Setting of Learning Task</h2>

In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 25
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

<h3>Initialize Workers</h3>

In [5]:
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

<h1>Load and Distribute Dataset</h1>

In [4]:
def Load_Data():
    
    '''Load CIFAR dataset from torch vision module distribute to workers using PySyft's Federated Data loader'''
    

    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    return federated_train_loader,test_loader

<h1>Define Neural Network Model</h1>

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    

<h1>Train Function</h1>

In [8]:

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

<h1>Test Function</h1>

In [9]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

<h1>Train Neural Network</h1>

In [10]:
#Load federated training data and test data
federated_train_loader,test_loader=Load_Data()

#Create Neural Network model instance
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

#Train Neural network and validate with test set after completion of training every epoch
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")

/Users/hrishikesh/anaconda3/envs/syft_flstm/lib/python3.6/site-packages/syft-0.1.3a1-py3.6.egg/syft/frameworks/torch/tensors/interpreters/native.py:216: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/50048 (0%)]	Loss: 2.310682
Train Epoch: 1 [640/50048 (1%)]	Loss: 2.302472
Train Epoch: 1 [1280/50048 (3%)]	Loss: 2.292667
Train Epoch: 1 [1920/50048 (4%)]	Loss: 2.299080
Train Epoch: 1 [2560/50048 (5%)]	Loss: 2.306766
Train Epoch: 1 [3200/50048 (6%)]	Loss: 2.297419
Train Epoch: 1 [3840/50048 (8%)]	Loss: 2.318182
Train Epoch: 1 [4480/50048 (9%)]	Loss: 2.309270
Train Epoch: 1 [5120/50048 (10%)]	Loss: 2.308285
Train Epoch: 1 [5760/50048 (12%)]	Loss: 2.295353
Train Epoch: 1 [6400/50048 (13%)]	Loss: 2.298305
Train Epoch: 1 [7040/50048 (14%)]	Loss: 2.302268
Train Epoch: 1 [7680/50048 (15%)]	Loss: 2.310489
Train Epoch: 1 [8320/50048 (17%)]	Loss: 2.302456
Train Epoch: 1 [8960/50048 (18%)]	Loss: 2.303723
Train Epoch: 1 [9600/50048 (19%)]	Loss: 2.294041
Train Epoch: 1 [10240/50048 (20%)]	Loss: 2.302730
Train Epoch: 1 [10880/50048 (22%)]	Loss: 2.296577
Train Epoch: 1 [11520/50048 (23%)]	Loss: 2.309041
Train Epoch: 1 [12160/50048 (24%)]	Loss: 2.301011
Train Epoch: 1 [12800/50048 

Train Epoch: 3 [3200/50048 (6%)]	Loss: 1.988908
Train Epoch: 3 [3840/50048 (8%)]	Loss: 1.932016
Train Epoch: 3 [4480/50048 (9%)]	Loss: 1.947524
Train Epoch: 3 [5120/50048 (10%)]	Loss: 1.896638
Train Epoch: 3 [5760/50048 (12%)]	Loss: 1.993257
Train Epoch: 3 [6400/50048 (13%)]	Loss: 2.017006
Train Epoch: 3 [7040/50048 (14%)]	Loss: 2.084772
Train Epoch: 3 [7680/50048 (15%)]	Loss: 1.909626
Train Epoch: 3 [8320/50048 (17%)]	Loss: 1.852309
Train Epoch: 3 [8960/50048 (18%)]	Loss: 2.100993
Train Epoch: 3 [9600/50048 (19%)]	Loss: 1.907568
Train Epoch: 3 [10240/50048 (20%)]	Loss: 1.918802
Train Epoch: 3 [10880/50048 (22%)]	Loss: 1.773605
Train Epoch: 3 [11520/50048 (23%)]	Loss: 1.858274
Train Epoch: 3 [12160/50048 (24%)]	Loss: 2.051283
Train Epoch: 3 [12800/50048 (26%)]	Loss: 2.055728
Train Epoch: 3 [13440/50048 (27%)]	Loss: 1.848284
Train Epoch: 3 [14080/50048 (28%)]	Loss: 1.964190
Train Epoch: 3 [14720/50048 (29%)]	Loss: 1.870093
Train Epoch: 3 [15360/50048 (31%)]	Loss: 1.917590
Train Epoch: 3

Train Epoch: 5 [6400/50048 (13%)]	Loss: 1.798672
Train Epoch: 5 [7040/50048 (14%)]	Loss: 1.690790
Train Epoch: 5 [7680/50048 (15%)]	Loss: 1.727154
Train Epoch: 5 [8320/50048 (17%)]	Loss: 1.872390
Train Epoch: 5 [8960/50048 (18%)]	Loss: 1.935997
Train Epoch: 5 [9600/50048 (19%)]	Loss: 1.597222
Train Epoch: 5 [10240/50048 (20%)]	Loss: 1.744767
Train Epoch: 5 [10880/50048 (22%)]	Loss: 1.715051
Train Epoch: 5 [11520/50048 (23%)]	Loss: 1.661238
Train Epoch: 5 [12160/50048 (24%)]	Loss: 1.708400
Train Epoch: 5 [12800/50048 (26%)]	Loss: 1.673660
Train Epoch: 5 [13440/50048 (27%)]	Loss: 1.691160
Train Epoch: 5 [14080/50048 (28%)]	Loss: 1.609215
Train Epoch: 5 [14720/50048 (29%)]	Loss: 1.715214
Train Epoch: 5 [15360/50048 (31%)]	Loss: 1.854467
Train Epoch: 5 [16000/50048 (32%)]	Loss: 1.837788
Train Epoch: 5 [16640/50048 (33%)]	Loss: 1.633036
Train Epoch: 5 [17280/50048 (35%)]	Loss: 1.546898
Train Epoch: 5 [17920/50048 (36%)]	Loss: 1.290452
Train Epoch: 5 [18560/50048 (37%)]	Loss: 1.511490
Train 

Train Epoch: 7 [9600/50048 (19%)]	Loss: 1.468385
Train Epoch: 7 [10240/50048 (20%)]	Loss: 1.723812
Train Epoch: 7 [10880/50048 (22%)]	Loss: 1.330731
Train Epoch: 7 [11520/50048 (23%)]	Loss: 1.387711
Train Epoch: 7 [12160/50048 (24%)]	Loss: 1.305721
Train Epoch: 7 [12800/50048 (26%)]	Loss: 1.584762
Train Epoch: 7 [13440/50048 (27%)]	Loss: 1.382669
Train Epoch: 7 [14080/50048 (28%)]	Loss: 1.507267
Train Epoch: 7 [14720/50048 (29%)]	Loss: 1.495025
Train Epoch: 7 [15360/50048 (31%)]	Loss: 1.485137
Train Epoch: 7 [16000/50048 (32%)]	Loss: 1.263469
Train Epoch: 7 [16640/50048 (33%)]	Loss: 1.383759
Train Epoch: 7 [17280/50048 (35%)]	Loss: 1.661530
Train Epoch: 7 [17920/50048 (36%)]	Loss: 1.727819
Train Epoch: 7 [18560/50048 (37%)]	Loss: 1.479962
Train Epoch: 7 [19200/50048 (38%)]	Loss: 1.429197
Train Epoch: 7 [19840/50048 (40%)]	Loss: 1.756681
Train Epoch: 7 [20480/50048 (41%)]	Loss: 1.547403
Train Epoch: 7 [21120/50048 (42%)]	Loss: 1.392619
Train Epoch: 7 [21760/50048 (43%)]	Loss: 1.499040
T

...

Train Epoch: 17 [18560/50048 (37%)]	Loss: 1.163087
Train Epoch: 17 [19200/50048 (38%)]	Loss: 1.187579
Train Epoch: 17 [19840/50048 (40%)]	Loss: 0.905330
Train Epoch: 17 [20480/50048 (41%)]	Loss: 1.240485
Train Epoch: 17 [21120/50048 (42%)]	Loss: 1.109342
Train Epoch: 17 [21760/50048 (43%)]	Loss: 0.857766
Train Epoch: 17 [22400/50048 (45%)]	Loss: 1.078485
Train Epoch: 17 [23040/50048 (46%)]	Loss: 1.362143
Train Epoch: 17 [23680/50048 (47%)]	Loss: 1.398843
Train Epoch: 17 [24320/50048 (49%)]	Loss: 1.045503
Train Epoch: 17 [24960/50048 (50%)]	Loss: 1.275162
Train Epoch: 17 [25600/50048 (51%)]	Loss: 1.324496
Train Epoch: 17 [26240/50048 (52%)]	Loss: 1.136015
Train Epoch: 17 [26880/50048 (54%)]	Loss: 1.131232
Train Epoch: 17 [27520/50048 (55%)]	Loss: 1.119464
Train Epoch: 17 [28160/50048 (56%)]	Loss: 1.097540
Train Epoch: 17 [28800/50048 (58%)]	Loss: 1.081855
Train Epoch: 17 [29440/50048 (59%)]	Loss: 0.913624
Train Epoch: 17 [30080/50048 (60%)]	Loss: 1.276978
Train Epoch: 17 [30720/50048 (6

Train Epoch: 19 [19840/50048 (40%)]	Loss: 1.021379
Train Epoch: 19 [20480/50048 (41%)]	Loss: 0.907784
Train Epoch: 19 [21120/50048 (42%)]	Loss: 1.050606
Train Epoch: 19 [21760/50048 (43%)]	Loss: 1.352780
Train Epoch: 19 [22400/50048 (45%)]	Loss: 1.147046
Train Epoch: 19 [23040/50048 (46%)]	Loss: 0.986722
Train Epoch: 19 [23680/50048 (47%)]	Loss: 0.924905
Train Epoch: 19 [24320/50048 (49%)]	Loss: 1.150382
Train Epoch: 19 [24960/50048 (50%)]	Loss: 1.039050
Train Epoch: 19 [25600/50048 (51%)]	Loss: 0.781595
Train Epoch: 19 [26240/50048 (52%)]	Loss: 1.355524
Train Epoch: 19 [26880/50048 (54%)]	Loss: 1.340935
Train Epoch: 19 [27520/50048 (55%)]	Loss: 1.074397
Train Epoch: 19 [28160/50048 (56%)]	Loss: 1.103497
Train Epoch: 19 [28800/50048 (58%)]	Loss: 1.109110
Train Epoch: 19 [29440/50048 (59%)]	Loss: 1.005160
Train Epoch: 19 [30080/50048 (60%)]	Loss: 1.018842
Train Epoch: 19 [30720/50048 (61%)]	Loss: 1.107206
Train Epoch: 19 [31360/50048 (63%)]	Loss: 0.881756
Train Epoch: 19 [32000/50048 (6

Train Epoch: 21 [21120/50048 (42%)]	Loss: 1.258770
Train Epoch: 21 [21760/50048 (43%)]	Loss: 1.015276
Train Epoch: 21 [22400/50048 (45%)]	Loss: 0.895441
Train Epoch: 21 [23040/50048 (46%)]	Loss: 1.149037
Train Epoch: 21 [23680/50048 (47%)]	Loss: 1.173993
Train Epoch: 21 [24320/50048 (49%)]	Loss: 1.055149
Train Epoch: 21 [24960/50048 (50%)]	Loss: 1.086061
Train Epoch: 21 [25600/50048 (51%)]	Loss: 0.997873
Train Epoch: 21 [26240/50048 (52%)]	Loss: 1.105449
Train Epoch: 21 [26880/50048 (54%)]	Loss: 1.047619
Train Epoch: 21 [27520/50048 (55%)]	Loss: 0.772701
Train Epoch: 21 [28160/50048 (56%)]	Loss: 0.914917
Train Epoch: 21 [28800/50048 (58%)]	Loss: 1.028080
Train Epoch: 21 [29440/50048 (59%)]	Loss: 1.252530
Train Epoch: 21 [30080/50048 (60%)]	Loss: 1.262798
Train Epoch: 21 [30720/50048 (61%)]	Loss: 1.342782
Train Epoch: 21 [31360/50048 (63%)]	Loss: 0.947154
Train Epoch: 21 [32000/50048 (64%)]	Loss: 1.155075
Train Epoch: 21 [32640/50048 (65%)]	Loss: 1.226779
Train Epoch: 21 [33280/50048 (6

Train Epoch: 23 [22400/50048 (45%)]	Loss: 1.033608
Train Epoch: 23 [23040/50048 (46%)]	Loss: 1.133567
Train Epoch: 23 [23680/50048 (47%)]	Loss: 1.275065
Train Epoch: 23 [24320/50048 (49%)]	Loss: 0.630646
Train Epoch: 23 [24960/50048 (50%)]	Loss: 0.734603
Train Epoch: 23 [25600/50048 (51%)]	Loss: 1.427874
Train Epoch: 23 [26240/50048 (52%)]	Loss: 1.300535
Train Epoch: 23 [26880/50048 (54%)]	Loss: 1.039309
Train Epoch: 23 [27520/50048 (55%)]	Loss: 0.842154
Train Epoch: 23 [28160/50048 (56%)]	Loss: 1.011201
Train Epoch: 23 [28800/50048 (58%)]	Loss: 0.990522
Train Epoch: 23 [29440/50048 (59%)]	Loss: 1.141753
Train Epoch: 23 [30080/50048 (60%)]	Loss: 1.102539
Train Epoch: 23 [30720/50048 (61%)]	Loss: 0.848370
Train Epoch: 23 [31360/50048 (63%)]	Loss: 1.034226
Train Epoch: 23 [32000/50048 (64%)]	Loss: 1.162783
Train Epoch: 23 [32640/50048 (65%)]	Loss: 0.657987
Train Epoch: 23 [33280/50048 (66%)]	Loss: 0.973812
Train Epoch: 23 [33920/50048 (68%)]	Loss: 1.035172
Train Epoch: 23 [34560/50048 (6

Train Epoch: 25 [23680/50048 (47%)]	Loss: 0.785874
Train Epoch: 25 [24320/50048 (49%)]	Loss: 1.106849
Train Epoch: 25 [24960/50048 (50%)]	Loss: 1.166161
Train Epoch: 25 [25600/50048 (51%)]	Loss: 1.099425
Train Epoch: 25 [26240/50048 (52%)]	Loss: 0.813766
Train Epoch: 25 [26880/50048 (54%)]	Loss: 1.132538
Train Epoch: 25 [27520/50048 (55%)]	Loss: 1.219898
Train Epoch: 25 [28160/50048 (56%)]	Loss: 0.767928
Train Epoch: 25 [28800/50048 (58%)]	Loss: 1.135707
Train Epoch: 25 [29440/50048 (59%)]	Loss: 0.985497
Train Epoch: 25 [30080/50048 (60%)]	Loss: 1.193842
Train Epoch: 25 [30720/50048 (61%)]	Loss: 0.978329
Train Epoch: 25 [31360/50048 (63%)]	Loss: 0.771826
Train Epoch: 25 [32000/50048 (64%)]	Loss: 1.071860
Train Epoch: 25 [32640/50048 (65%)]	Loss: 0.931440
Train Epoch: 25 [33280/50048 (66%)]	Loss: 0.715828
Train Epoch: 25 [33920/50048 (68%)]	Loss: 0.964404
Train Epoch: 25 [34560/50048 (69%)]	Loss: 1.246645
Train Epoch: 25 [35200/50048 (70%)]	Loss: 1.305120
Train Epoch: 25 [35840/50048 (7